## Example of cluster tree usage

### Installation (if needed)

**Note**: replace cupy-cuda110 with your cuda version!!!

In [1]:
# !pip install cupy-cuda110

In [2]:
# !pip uninstall py-boost
# !cd ..; bash build_package.sh
# !pip install ../dist/Py_Boost-0.1.8-py3-none-any.whl

### Imports

In [3]:
import os
# Optional: set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.makedirs('../data', exist_ok=True)

import joblib
import numpy as np
from sklearn.datasets import make_regression

# simple case - just one class is used
from py_boost import GradientBoosting

### Generation of dummy regression data

In [4]:
%%time
X, y = make_regression(150000, 100, n_targets=1, random_state=42)
X_test, y_test = X[:50000], y[:50000]
X, y = X[-50000:], y[-50000:]

CPU times: user 2.06 s, sys: 1.29 s, total: 3.35 s
Wall time: 826 ms


### Training a GBDT model

The only argument required here is a loss function. It, together with the input target shape, determines the task type. The loss function can be passed as a Loss instance or using a string alias:

* ***'mse'*** for the regression/multitask regression
* ***'msle'*** for the regression/multitask regression
* ***'bce'*** for the binary/multilabel classification
* ***'crossentropy'*** for the multiclassification

Training is simply done by calling the .fit metod. Possible argumentsare the following:

* ***'X'*** 
* ***'y'*** 
* ***'sample_weight'*** 
* ***'eval_sets'***  
A validation set is passed as a list of dicts with possible keys ['X', 'y', 'sample_weight']. Note: if multiple valid sets are passed, the best model is selected using the last one.

#### The example below illustrates how to train a simple regression task.

In [5]:
%%time
model = GradientBoosting('mse', ntrees=1000, verbose=100)

model.fit(X, y, eval_sets=[{'X': X_test, 'y': y_test},])

[19:51:30] Stdout logging level is INFO.
[19:51:30] GDBT train starts. Max iter 1000, early stopping rounds 100
[19:51:30] Iter 0; Sample 0, rmse = 175.085401707104; 
[19:51:31] Iter 100; Sample 0, rmse = 34.704968699444635; 
[19:51:32] Iter 200; Sample 0, rmse = 18.87524749221564; 
[19:51:33] Iter 300; Sample 0, rmse = 15.978965345652862; 
[19:51:34] Iter 400; Sample 0, rmse = 15.373245716125354; 
[19:51:35] Iter 500; Sample 0, rmse = 15.218444378728087; 
[19:51:36] Iter 600; Sample 0, rmse = 15.164194374641644; 
[19:51:36] Iter 700; Sample 0, rmse = 15.094284385839273; 
[19:51:37] Iter 800; Sample 0, rmse = 15.029151646091728; 
[19:51:38] Iter 900; Sample 0, rmse = 14.96591336234754; 
[19:51:39] Iter 999; Sample 0, rmse = 14.916066868203313; 
CPU times: user 11.8 s, sys: 818 ms, total: 12.6 s
Wall time: 11.3 s


Here we will make prediction from different stages and will try to cluster paths

In [6]:
%%time
iters_to_pred = np.arange(45).cumsum()

test_staged_pred = model.predict_staged(X_test, iterations=iters_to_pred)
# shape - (iterations, objects, output_dim)
test_staged_pred.shape

CPU times: user 881 ms, sys: 314 ms, total: 1.2 s
Wall time: 1.21 s


(45, 50000, 1)

Calculate errors

In [7]:
error_paths = ((y_test[np.newaxis, :] - test_staged_pred[..., 0]) ** 2).T
error_paths[0]

array([7.76091595e+03, 7.91277798e+03, 6.88692004e+03, 6.62394329e+03,
       5.07055708e+03, 2.98405447e+03, 1.90369454e+03, 1.42643350e+03,
       1.02466591e+03, 5.42772313e+02, 2.73758818e+02, 1.03262352e+02,
       7.15182801e+00, 5.33727182e+00, 2.04351677e+01, 1.29328979e+01,
       9.64889161e+00, 1.40341080e+01, 9.99066560e+00, 1.22498699e+01,
       1.40067976e+01, 1.84562880e+01, 1.60255573e+01, 1.60130986e+01,
       1.84176320e+01, 1.70986568e+01, 1.78987097e+01, 1.90122375e+01,
       2.24470121e+01, 2.29832696e+01, 2.36874766e+01, 2.51050573e+01,
       2.08873772e+01, 1.90738303e+01, 1.98194901e+01, 1.88469976e+01,
       1.73786778e+01, 2.01084945e+01, 2.08516176e+01, 2.05897623e+01,
       2.08362914e+01, 2.09833055e+01, 1.89430398e+01, 1.98024432e+01,
       2.25277642e+01])

Create cluster tree

In [8]:
from py_boost.gpu.cluster_tree import ClusterCandidates

clustering = ClusterCandidates(depth_range=list(range(1, 7)), min_data_in_leaf=100)

In [ ]:
%%time
clustering.fit(X_test, error_paths)

In [9]:
cluster_candidates = clustering.predict(X_test)
# shape - (n_candidates, n_objects, 1)
cluster_candidates.shape

(50000, 6)

In [10]:
cluster_candidates

array([[0, 2, 3, 4, 6, 9],
       [0, 2, 3, 4, 6, 9],
       [0, 2, 3, 4, 6, 9],
       ...,
       [0, 2, 2, 3, 5, 8],
       [0, 2, 2, 3, 4, 6],
       [0, 2, 3, 4, 6, 9]], dtype=uint32)

In [11]:
cluster_candidates.max(axis=0)

array([ 1,  2,  3,  5,  7, 10], dtype=uint32)